In [2]:
# Imports and vars
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import cred

In [3]:
# Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id= cred.client_id, client_secret= cred.client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [46]:
top_globals_uri = "37i9dQZEVXbNG2KDcFcKOF?si=77d8f5cd51cd478d"
best_of_all_uri = "6PPayXaPylORShOqea9n9S?si=94e26f660488407b"
disney_test_uri = "1uaHLzRPa406tDisDEeDFd?si=42f5f7306a554cd7"

# track["track"].keys() = ['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit',
#'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number',
#'type', 'uri']
#for track in sp.playlist_tracks(top_globals_uri)["tracks"]["items"]:
#    print(track["track"]["name"])
    
for track in sp.playlist_tracks(disney_test_uri)["tracks"]["items"]:
    print(track["track"]["name"] + " - " + track["track"]["album"]["name"])

Quando a Minha Vida Vai Começar? - Enrolados (Trilha Sonora Original)
No Verão - Frozen: Uma Aventura Congelante (Trilha Sonora Original)
Uma Vez Na Eternidade - Frozen: Uma Aventura Congelante (Trilha Sonora Original)
Vejo Enfim a Luz Brilhar - Enrolados (Trilha Sonora Original)
Alguma Coisa Acontece - A Bela e A Fera (Trilha Sonora Original em Português)
Bela (Reprise) - A Bela e A Fera (Trilha Sonora Original em Português)
Reparos - Frozen: Uma Aventura Congelante (Trilha Sonora Original)
Saber Quem Sou - Moana: um mar de aventuras (Trilha sonora original em português)
Lá em Nova Orleans (Down in New Orleans) - Princesas Disney 2
Encanto da Cura - Enrolados (Trilha Sonora Original)
De nada - Moana: um mar de aventuras (Trilha sonora original em português)
Bela - A Bela e A Fera (Trilha Sonora Original em Português)
Rena É Melhor do Que Gente - Frozen: Uma Aventura Congelante (Trilha Sonora Original)
Um Mundo Ideal (A Whole New World) - Eliana 2001
A Noite Da Arábia (2019) - Aladdin 

In [2]:
# From a category and ID, a uri is generated
def generate_uri(category, id):
    return "spotify:" + category + ":" + id